In [16]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
from selenium import webdriver
import time
import pandas as pd
import os
import requests

In [2]:
# Import splinter and set the cromedriver path
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# Set window size to ensure all of the elements needed are displayed
# Width, Height
browser.driver.set_window_size(640, 480)

# Scrape Reddit for Subscribers Count - Conner

In [ ]:
subreddits = ["AZCardinals", "falcons", "ravens", "buffalobills", "panthers","CHIBears", "bengals", "Browns",
              "cowboys", "DenverBroncos", "detroitlions", "GreenBayPackers", "Texans", "Colts", "Jaguars",
              "KansasCityChiefs", "miamidolphins", "minnesotavikings", "Patriots", "Saints", "NYGiants", "nyjets",
              "oaklandraiders", "eagles", "steelers", "LosAngelesRams", "Chargers", "49ers", "Seahawks", 
              "buccaneers", "Tennesseetitans", "Redskins"]

subcounts = []

In [ ]:
for sub in subreddits:
    time.sleep(1)
    url = f"https://www.reddit.com/r/{sub}"
    browser.visit(url)
    time.sleep(1)
    html = browser.html
    soup = bs(html, "html.parser")
    subscribers = int(float(soup.find("p", class_="s1bd5ppi-10").text.split("k")[0])*1000)
    dictionary = {"Subreddit URL (https://www.reddit.com/r/)":sub, "Subs":subscribers}
    subcounts.append(dictionary)

In [ ]:
subcounts

In [ ]:
df = pd.DataFrame(subcounts)
teams = pd.read_csv("Teams.csv")

In [ ]:
combined = pd.merge(df, teams, on="Subreddit URL (https://www.reddit.com/r/)")
combined.head()

In [ ]:
combined = combined.drop(["Subreddit URL (https://www.reddit.com/r/)"], axis=1)
combined.head()

In [ ]:
combined.to_csv("TeamsSubs.csv", index=False, header=True)

# Scrape Site for Valuation - Damita

In [22]:
# Scarpe Forbes (F) Site for Team Valuations
F_url = 'https://www.forbes.com/nfl-valuations/list/#tab:overall'
F_response = requests.get(F_url)
F_soup = bs(F_response.text, 'lxml')
#print(F_soup.prettify())

In [32]:
# OPtion 1:  Visit the Forbes webpage and use Pandas to scrape the table containing team data
team_facts_url = 'https://www.forbes.com/nfl-valuations/list/#tab:overall'
tables = pd.read_html(team_facts_url)
tables

[Empty DataFrame
 Columns: [Unnamed: 0, Rank, Team, Current ValueValue of team based on current stadium deal (unless new stadium is pending), without deduction for debt (other than stadium debt)., 1-Yr Value Change, Debt/ValueIncludes stadium debt, Revenue, Operating IncomeEarnings before interest, taxes, depreciation and amortization.]
 Index: []]

In [42]:
#OPTION 2: Parse through table for team name and value
F_page_result = F_soup.find('tbody', id_='list_table_body')
#F_team_name = F_page_result.find('td', attrs={'class': 'name'})
#F_team_data = F_page_result.find('tr', class_='data')
F_team_name = F_page_result.find('td', class_='name').find('a').text
#F_team_name = F_page_result.find('td', attrs={'class': 'name'})
F_team_name

AttributeError: 'NoneType' object has no attribute 'find'

In [ ]:
# Create a dictionary to hold valuations
value = {}
value[team_name]
value[team_value]